In [21]:
from llama_index.llms.ollama import Ollama

llm = Ollama(model="deepseek-r1:8b")

In [1]:
from llama_index.readers.file import UnstructuredReader
from pathlib import Path

subjects = ['Biology', 'Chemistry', 'Physics', 'Mathematics', 'Computer_science']

loader = UnstructuredReader()
doc_set = {}
all_docs = []
for subject in subjects:
    subject_docs = loader.load_data(
        file=Path(f"./data/{subject}.pdf"), split_documents=False
    )
    # insert year metadata into each year
    for s in subject_docs:
        s.metadata = {"subject": subject}
    doc_set[subject] = subject_docs
    all_docs.extend(subject_docs)

c:\Users\adria\anaconda3\envs\llm_env_3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from llama_index.core import VectorStoreIndex, StorageContext, Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# Configure document chunking size
Settings.chunk_size = 512

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

index_set = {}
for subject in subjects:
    storage_context = StorageContext.from_defaults()
    cur_index = VectorStoreIndex.from_documents(
        doc_set[subject],
        storage_context=storage_context,
        embed_model=embed_model,  # use the open source embeddings model
    )
    index_set[subject] = cur_index
    storage_context.persist(persist_dir=f"./storage/{subject}")


In [3]:
# Load indices from disk
from llama_index.core import load_index_from_storage

index_set = {}
for subject in subjects:
    storage_context = StorageContext.from_defaults(
        persist_dir=f"./storage/{subject}"
    )
    cur_index = load_index_from_storage(
        storage_context,
        embed_model=embed_model,
    )
    index_set[subject] = cur_index

In [22]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

individual_query_engine_tools = [
    QueryEngineTool(
        query_engine=index_set[subject].as_query_engine(llm=llm),
        metadata=ToolMetadata(
            name=f"vector_index_{subject}",
            description=f"useful for when you want to answer queries about the {subject} subject",
        ),
    )
    for subject in subjects
]

In [23]:
from llama_index.core.query_engine import SubQuestionQueryEngine

query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=individual_query_engine_tools,
    llm=llm,
)

In [24]:
query_engine_tool = QueryEngineTool(
    query_engine=query_engine,
    metadata=ToolMetadata(
        name="sub_question_query_engine",
        description="useful for when you want to answer queries that require analyzing multiple subjects",
    ),
)

In [25]:
tools = individual_query_engine_tools + [query_engine_tool]

In [26]:
# Define a custom prompt template to enforce the chain-of-thought format
custom_prompt = """
You are an intelligent agent that answers queries by following a strict chain-of-thought format.
For every step, please output your reasoning and your tool calls using exactly the following structure:

Thought: <Your internal reasoning for this step.>
Action: <The tool name you want to use (or output "None" if no tool is used).>
Action Input: <The JSON input to pass to the tool (or "{}" if no tool is used).>

When you are ready to provide your final answer, output only:
Answer: <Your final answer here.>

Do not output any extra text or commentary outside of this structure.
"""

In [27]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(tools, llm=llm, verbose=True, prompt_template=custom_prompt)

In [28]:
while True:
    text_input = input("User: ")
    if text_input == "exit":
        break
    response = agent.chat(text_input)
    print(f"Agent: {response}")

> Running step 5d3ee4f6-b3dd-4269-8157-3e4ff7dcc064. Step input: hello


ResponseError: llama runner: failed to load model 'C:\Users\adria\.ollama\models\blobs\sha256-6340dc3229b0d08ea9cc49b75d4098702983e17b4c096d57afbbf2ffc813f2be': this model may be incompatible with your version of Ollama. If you previously pulled this model, try updating it by running `ollama pull deepseek-r1:8b` (status code: 500)